Load movies metadata dataset into a pandas DataFrame (These are not the Data from Netflix!!!)

In [2]:
# code
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
  
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

Movie Datei und Combined Data einbinden

In [ ]:
ratings = pd.read_csv("C:/Users/passi/OneDrive - Hochschule Osnabrück/Semester 5/Programmierprojekt/Cobined_data_new_v1.csv")
ratings = ratings.replace(';', ',')
ratings.head()

C:\Users\passi\AppData\Local\Temp\ipykernel_14944\2497152730.py:1: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  ratings = pd.read_csv("C:/Users/passi/OneDrive - Hochschule Osnabrück/Semester 5/Programmierprojekt/Cobined_data_new.csv")


,Cust_Id,Rating,Date,Movie
0,1488844,3.0,2005-09-06,1.0
1,822109,5.0,2005-05-13,1.0
2,885013,4.0,2005-10-19,1.0
3,30878,4.0,2005-12-26,1.0
4,823519,3.0,2004-05-03,1.0


In [3]:
movies = pd.read_csv("C:/Users/passi/OneDrive - Hochschule Osnabrück/Semester 5/Programmierprojekt/movie_titles_new.csv")
movies = ratings.replace('	', ',')
movies.head()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 221562: invalid continuation byte

Ein paar Datenanalysen (Für RecSys nicht wichtig)

In [6]:
n_ratings = len(ratings)
n_movies = len(ratings['Movie'].unique())
n_users = len(ratings['Cust_Id'].unique())
  
print(f"Number of ratings: {n_ratings}")
print(f"Number of unique movieId's: {n_movies}")
print(f"Number of unique users: {n_users}")
print(f"Average ratings per user: {round(n_ratings/n_users, 2)}")
print(f"Average ratings per movie: {round(n_ratings/n_movies, 2)}")
  
user_freq = ratings[['Cust_Id', 'Movie']].groupby('Cust_Id').count().reset_index()
user_freq.columns = ['Cust_Id', 'n_ratings']
user_freq.head()

Number of ratings: 100836
Number of unique movieId's: 9724
Number of unique users: 610
Average ratings per user: 165.3
Average ratings per movie: 10.37


,userId,n_ratings
0,1,232
1,2,29
2,3,39
3,4,216
4,5,44


In [7]:
# Find Lowest and Highest rated movies:
mean_rating = ratings.groupby('Movie')[['rating']].mean()
# Lowest rated movies
lowest_rated = mean_rating['rating'].idxmin()
movies.loc[movies['Movie'] == lowest_rated]
# Highest rated movies
highest_rated = mean_rating['rating'].idxmax()
movies.loc[movies['Movie'] == highest_rated]
# show number of people who rated movies rated movie highest
ratings[ratings['Movie']==highest_rated]
# show number of people who rated movies rated movie lowest
ratings[ratings['Movie']==lowest_rated]
  
## the above movies has very low dataset. We will use bayesian average
movie_stats = ratings.groupby('Movie')[['rating']].agg(['count', 'mean'])
movie_stats.columns = movie_stats.columns.droplevel()

Matrix erstellen

In [8]:
# Now, we create user-item matrix using scipy csr matrix
from scipy.sparse import csr_matrix
  
def create_matrix(df):
      
    N = len(df['Cust_Id'].unique())
    M = len(df['Movie'].unique())
      
    # Map Ids to indices
    user_mapper = dict(zip(np.unique(df["Cust_Id"]), list(range(N))))
    movie_mapper = dict(zip(np.unique(df["Movie"]), list(range(M))))
      
    # Map indices to IDs
    user_inv_mapper = dict(zip(list(range(N)), np.unique(df["Cust_Id"])))
    movie_inv_mapper = dict(zip(list(range(M)), np.unique(df["Movie"])))
      
    user_index = [user_mapper[i] for i in df['Cust_Id']]
    movie_index = [movie_mapper[i] for i in df['Movie']]
  
    X = csr_matrix((df["rating"], (movie_index, user_index)), shape=(M, N))
      
    return X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper
  
X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper = create_matrix(ratings)
  
from sklearn.neighbors import NearestNeighbors

KNN Methode anwenden

In [9]:
## Find similar movies using KNN
def find_similar_movies(movie_id, X, k, metric='cosine', show_distance=False):
      
    neighbour_ids = []
      
    movie_ind = movie_mapper[movie_id]
    movie_vec = X[movie_ind]
    k+=1
    kNN = NearestNeighbors(n_neighbors=k, algorithm="brute", metric=metric)
    kNN.fit(X)
    movie_vec = movie_vec.reshape(1,-1)
    neighbour = kNN.kneighbors(movie_vec, return_distance=show_distance)
    for i in range(0,k):
        n = neighbour.item(i)
        neighbour_ids.append(movie_inv_mapper[n])
    neighbour_ids.pop(0)
    return neighbour_ids
  
  
movie_titles = dict(zip(movies['Movie'], movies['title']))
  
movie_id = 3
  
similar_ids = find_similar_movies(movie_id, X, k=10)
movie_title = movie_titles[movie_id]
  
print(f"Since you watched {movie_title}")
for i in similar_ids:
    print(movie_titles[i])

Since you watched Grumpier Old Men (1995)
Grumpy Old Men (1993)
Striptease (1996)
Nutty Professor, The (1996)
Twister (1996)
Father of the Bride Part II (1995)
Broken Arrow (1996)
Bio-Dome (1996)
Truth About Cats & Dogs, The (1996)
Sabrina (1995)
Birdcage, The (1996)
